# Nice Magics

This section is dedicated to magics you technically could live without, but that you're better off having. Let's jump right in.

### `%pastebin` for sharing code

This is a really lovely feature for quickly sharing code cleanly/easily. Without `%pastebin`, you basically have two options, a screenshot (preserves prettiness, can't be copy/pasted) or copy/pasting code in a chat window (loses prettiness, can be copy/pasted). With `%pastebin` we get the best of both.

To use, just type `%pastebin n` where n is the cell you want to share. Say I wished to share my SignalShifting code in a way that could be read, and then copied if necessary  
![Signal shifting code example](images/pastebin_magic3.png)

Here `%pastebin 17` is copying the code from cell 17, and loading it to a website where it will last for 7 days, all I need to do is copy and paste that link in whatever chat client I want, and my friend will see this.
![Signal shifting code on pastebin](images/pastebin_magic2.png)

### `%env` to view/set environment variables

Nobody likes dealing with environment variables, but every once in a while it's necessary. The documentation on this magic is very well-written and concise.  
![Environment magic documentation](images/env_magic.png)  
Note that any changes you make will only persist for the current session, for how to setup permanent environment variables in Jupyter, see [this StackOverflow post](https://stackoverflow.com/a/53595397/5042053)

I've set up a series of commands to demonstrate how to use all of the above features

In [ ]:
# list all environment variables/values
%env?

In [ ]:
# try to look up a non-existent variable
%env newval

In [ ]:
# create and initialize a non-existent variable
%env newval = 42

In [ ]:
# now we can look it up
%env newval

In [ ]:
# let's try to set it to be equal to a python variable `x`
x = 7
%env newval=x

In [ ]:
# Oops, looks like it literally set it to the string x
%env newval

In [ ]:
# We need to use a dollar sign to trigger interpolation
%env newval=$x

In [ ]:
#Looks like that worked!
%env newval

### `%store` to pass values between notebooks

If you are developing a library using Jupyter Notebooks, you should be using [nbdev by fastai](https://nbdev.fast.ai/), but most of the time, I'm just hacking something together. I know better than to do it all in one giant notebook, but the problem then becomes, I have an output from one notebook, say a notebook for preprocessing, that I want to use as an input in another notebook. Sure, I could manually export it, or pickle it, but with `%store` there's a better way.  

All you do is call `%store varname` for the variable you want to export, and then `%store -r varname` to bring it to life in another notebook. To use `%store` effectively, finish each notebook by storing the data and values you would like to make available to future notebooks. 

I'll demonstrate below, but instead of making you open another notebook, we'll just delete the variable, demonstrate it's gone, and then bring it back to life. I encourage you to try it yourself across different notebooks.

In [ ]:
# make a dict {1:'a', 2:'b'...26:'z'} as a fake dataset 
def make_dataset():
    alpha = 'abcdefghijklmnopqrstuvwxyz'
    return {i+1:alpha[i] for i in range(26)}

In [ ]:
important_data = make_dataset()

In [ ]:
%store important_data

In [ ]:
del important_data

In [ ]:
important_data[13]

In [ ]:
%store -r important_data

In [ ]:
important_data[13]

#### How it works

Behind the scenes, `%store` is using pickle (a built in library for storing Python objects on disk) to save and load objects. That means you can use `%store` for any pickleable objects.  

Here's the minimum you need to know about pickle:
![What can be pickled and unpickled](images/pickle1.png)
[Source 1](https://docs.python.org/2/library/pickle.html#what-can-be-pickled-and-unpickled)
[Source 2](https://docs.python.org/3.8/library/pickle.html)

#### Sharing functions across notebooks

Unfortunately pickling notebook functions isn't as easy. There are a number of libraries to help do this like `nbimporter` but even the creator of that library [Gregor Sturm](https://github.com/grst) now [recommends against it's usage.](https://github.com/grst/nbimporter#update-2019-06-i-do-not-recommend-any-more-to-use-nbimporter) 

His solution, and one I use as well, is to make a `utils.py` file and refactor any functions used in multiple notebooks to be defined there. This is a solution that I only recommend for small projects like a kaggle competition, or a new idea you're experimenting with. If you have so many functions that copying them to a single file seems like a bad solution, use [nbdev by fastai](https://nbdev.fast.ai/)

#### Autoload everything you store NOTE NEEDS WORK

Note that this tip has potential for misuse. Dumping everything you've ever stored into the state each notebook you create seems like a bad idea in general, but may be appropriate for a few unique situations. 
`%config StoreMagics.autorestore=True`

Note: this setting doesnt persist, we need to adjust it in the config once I figure that part out. 

In [ ]:
%config StoreMagics.autorestore=True

### `%hist` for seeing execution history

`%hist` is an alias for `%history` that will show you all commands that have been executed since the kernel last restarted. It is properly indented and without line numbers as to be reproducible by copy/pasting to a file. 

Optional Arguments:
- A range of numbers e.g. `2-5` to see the 2nd to 5th most recent commands
- `-n` to see line numbers 
- `-o` to include outputs 
- `-t` to translate everything to valid python source code (e.g. `%hist` will appear as `get_ipython().run_line_magic('hist', '')` 

The last option is extremely cool as it allows us to demystify all the stuff Jupyter is doing for us under the hood. Let's take a look at how and how `?` and `??` actually work

In [ ]:
%hist?

In [ ]:
%hist??

In [ ]:
%hist -t 1-2

get_ipython().run_line_magic('pinfo', '%hist')
get_ipython().run_line_magic('pinfo2', '%hist')


This shows us that `?` and `??` are actually just convenient shortcuts to the magics `%pinfo` and `%pinfo2`, which we don't ever need to use because `?` and `??` are so much better!

### `%%javascript` and other language changing script magics

The focus of this book is using Jupyter for Python, but occasionally it is nice to have the ability to run code in other languages. For instance in Jupyter, much of the underlying functionality is written in Javascript, and being able to run javascript from the console allows us to do things like play around with settings or shortcuts to see if they work before we go add them to custom.js to make the change permanent.

I first learned about this from [Stas Bekman](https://forums.fast.ai/u/stas) in the excellent [Jupyter Notebook Tips and Tricks Thread](https://forums.fast.ai/t/jupyter-notebook-enhancements-tips-and-tricks/17064/2) on fastai forums. 

Below is a short example of some javascript that will manipulate Jupyter by executing the next cell, waiting 3 seconds, and inserting another cell below


In [ ]:
%%javascript
Jupyter.notebook.execute_cell()
setTimeout(function(){
    Jupyter.notebook.insert_cell_below()
}, 3000)

<IPython.core.display.Javascript object>

In [ ]:
print("Executed")

Executed


`%%javascript` is one of many language changing options. Others are
- `%%perl`
- `%%ruby`
- `%%js` (shortcut for javascript)
- `%%python` to run in the default python interpreter, or `%%python2`/`%%python3` for specific versions.
- `%%bash` - to run a cell in bash as a subprocess.\*

\* I'm not actually sure of the functional difference between `%%bash` and `!`, if you do, message me and I'll credit you here.

Using `?` with all of these gives no new info, because they rely on the `%%script` magic to run, so if you're having trouble getting one of these language magics to work, that's the best place to look for documentation. Finally, if you look at the list of cell magics using `%lsmagic`, you'll notice there are a number of other examples I left out here, including `%%latex`, `%%markdown`, and `%%html`, that's because Jupyter now will automatically recognize and render all 3 of those, and the magics are no longer needed.

### `%more` for quickly viewing files in the pager REPLACE WITH NON .PY FILE

Do you have a file you want to quickly examine without having to go back to the Jupyter navigation page and open it in a new tab? Use `%more filename`   

In [ ]:
%more error.py

### `%tb` to see your last error/stack-trace

tb is short for traceback. Sometimes you delete cells, or maybe you're in another part of the notebook and you want to see that error again. Bring it back to life with `%tb`

In [ ]:
%tb

UsageError: %%latex is a cell magic, but the cell body is empty.


In [ ]:
### 

### `%whos` to see details about local variables 

<div class="alert alert-block alert-warning"><strong>Note:</strong> If you often import libraries with <code>from foo import *</code>, this won't be useful as the namespace will be too cluttered by those imports, which will be included when using `%whos`</div>

`%whos` will print a nice little table to show you what variables are populating the current notebook's namespace. 
![Whos magic](images/whos_magic.png)

You can also pass in a type (e.g. list) as a filter, and it will only show you variables of that type.

There are two alternatives `%who`, which just prints the variable names, and `%who_ls` which gives you back a list of the variable names. I prefer `%whos` for the extra info it provides but I'd recommend trying all three to see what suites you best. 

In [ ]:
%who

No variables match your requested type.


In [ ]:
%who

NamespaceMagics	 get_ipython	 getsizeof	 json	 np	 var_dic_list	 


In [ ]:
%who_ls

['NamespaceMagics', 'get_ipython', 'getsizeof', 'json', 'np', 'var_dic_list']

#### Non-magic alternatives: Variable Explorer Extension

If you prefer a GUI, you can use the Variable Explorer Extension discussed in Chapter 2. While it won't be very useful if you use `from foo import *` it is a cool alternative to using `%whos`

#### Non-magic alternatives: `locals()` and `dir()`

Python offers us `locals()` and `dir()` but these will show variables Jupyter is using internally to track stuff like command history. Use that if you're interested in exploring, but if you just want to see what you've personally set, use `%whos`

In [ ]:
len(locals())

46

In [ ]:
len(dir())

48

In [ ]:
x = %who_ls
len(x)

7